In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import RepeatedKFold

In [ ]:
# Importar datos
Dataframe = pd.read_csv("WineQT.csv",sep=";")
Dataframe.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,6
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,8
9,6.7,0.58,0.08,1.8,0.097,15.0,65.0,0.9959,3.28,0.54,9.2,5,10


In [ ]:
Dataframe.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,pH,sulphates,quality,Id
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,3.311015,0.657708,5.657043,804.969379
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.156664,0.170399,0.805824,463.997116
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,2.740000,0.330000,3.000000,0.000000
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,3.205000,0.550000,5.000000,411.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,3.310000,0.620000,6.000000,794.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,3.400000,0.730000,6.000000,1209.500000
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,4.010000,2.000000,8.000000,1597.000000


In [ ]:
# En la ejecución anterior se detecta que las columnas 
# density y alcohol no son listadas, lo que implica alguna
# anomalia con el tipo de dato, se analiza este tipo ahora
print(Dataframe.dtypes)


fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                  object
pH                      float64
sulphates               float64
alcohol                  object
quality                   int64
Id                        int64
dtype: object


In [ ]:
# Dado que son de tipo object, se transforma a 
# un tipo de dato float64
alcohol_ = []
density_ = []
for i in range(Dataframe.shape[0]):
  try: #Se usa el try por que existen algunos pocos valores anormales que python no pudo transformar
    temp_alcohol = float(Dataframe['alcohol'][i])
  except:
    temp_alcohol = None #Los valores que no son posibles se rellenan con NAN
  alcohol_.append(temp_alcohol)

for i in range(Dataframe.shape[0]):
  try:
    temp_density = float(Dataframe['density'][i])
  except:
    temp_density = None
  density_.append(temp_density)

Dataframe['Alcohol_'] = alcohol_
Dataframe['Density_'] = density_

In [ ]:
# Verificar que ahora todas las columnas son númericas
Dataframe.describe()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,pH,sulphates,quality,Id,Alcohol_,Density_
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1138.000000,1088.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,3.311015,0.657708,5.657043,804.969379,10.440598,0.996511
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.156664,0.170399,0.805824,463.997116,1.079716,0.001685
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,2.740000,0.330000,3.000000,0.000000,8.400000,0.990070
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,3.205000,0.550000,5.000000,411.000000,9.500000,0.995500
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,3.310000,0.620000,6.000000,794.000000,10.200000,0.996560
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,3.400000,0.730000,6.000000,1209.500000,11.100000,0.997600
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,4.010000,2.000000,8.000000,1597.000000,14.900000,1.000000


In [ ]:
# Ahora que se relleno con Nan, estos valores se rellenan con la media
Alcohol_Filled = Dataframe["Alcohol_"].fillna(Dataframe["Alcohol_"].mean()) #Creación  de columnas con relleno
Density_Filled = Dataframe["Density_"].fillna(Dataframe["Density_"].mean()) #Creación  de columnas con relleno
# Ahora se asignan las columnas finales que usaremos
Dataframe['Density_Filled'] = Density_Filled
Dataframe['Alcohol_Filled'] = Alcohol_Filled

In [ ]:
# Como no existen datos nulos y todas las 
# caracteristicas son numericas no hace falta
# codificar, limpiar o rellenar valores
# Ahora se dividen las etiquetas de las caracterticas
X = Dataframe[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
               'chlorides',	'free sulfur dioxide', 'total sulfur dioxide',	'Density_Filled',	
               'pH',	'sulphates',	'Alcohol_Filled']].values #Adquisición de caracteristicas
y = Dataframe['quality'].values #Adquisición de etiquetas 

In [ ]:
#División del Dataset con 10% para test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
#Normalización de los datos
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Aplicar PCA
pca = decomposition.PCA(whiten=False,svd_solver='auto')
pca.fit(X_train)
X_trainPCA = pca.transform(X_train) 

print("Pesos aplicando PCA:", pca.explained_variance_ratio_)

Pesos aplicando PCA: [0.27738702 0.17118853 0.14631761 0.10744274 0.08507441 0.06204267
 0.05069461 0.0399921  0.03135792 0.01647125 0.01203113]


In [ ]:
# Se puede reducir de 11 a 9 caracteristicas
# 97.15% 
pca = decomposition.PCA(n_components=9)
pca.fit(X_train)

PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
PCA_X = pca.transform(X)

In [ ]:
# KNN iterando hiperparametros y usando crossvalidation de 5 pliegues 
parametrosKNN = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 'weights':('uniform', 'distance'),'metric':('manhattan','euclidean','minkowski', 'chebyshev')}
Clasificador_KNN = KNeighborsClassifier()
#refit=True, el reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en .best_estimator_.
clf = GridSearchCV(Clasificador_KNN, parametrosKNN,cv=5,scoring='accuracy',refit=True , return_train_score=False,verbose=1)
clf.fit(PCA_X_train, y_train);
print("Los mejores hiperparámetros encontrados" + str(clf.best_params_) + " score: " + str(clf.best_score_*100),"%")
modelo_finall = clf.best_estimator_
weighted_prediction = modelo_finall.predict(PCA_X_test)
print('MCC = ',matthews_corrcoef(y_test,modelo_finall.predict(PCA_X_test))*100,"%")
print('F1 score:', f1_score(y_test, weighted_prediction,average='weighted')*100,"%")


Fitting 5 folds for each of 160 candidates, totalling 800 fits
Los mejores hiperparámetros encontrados{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'} score: 64.0094719393796 %
MCC =  55.1704329617036 %
F1 score: 70.98672347066824 %


In [ ]:
#Gaussian Naive Bayes, usando crossvalidation de 5 pliegues
params = {            
            'var_smoothing': [1e-9, 1e-6, 1e-12],
         }

gaussian_nb_grid = GridSearchCV(GaussianNB(), param_grid=params, refit=True, n_jobs=-1, cv=5, verbose=5)
gaussian_nb_grid.fit(PCA_X_train, y_train)
modelo_finall2 = gaussian_nb_grid.best_estimator_

weighted_prediction3 = modelo_finall2.predict(PCA_X_test)
print("Los mejores hiperparámetros encontrados" + str(gaussian_nb_grid.best_params_) + " score: " + str(gaussian_nb_grid.best_score_*100),"%")
print('MCC = ',matthews_corrcoef(y_test,modelo_finall2.predict(PCA_X_test))*100,"%")
print("F1 SCORE = ",f1_score(y_test,weighted_prediction3,average='weighted')*100,"%")


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Los mejores hiperparámetros encontrados{'var_smoothing': 1e-09} score: 55.15273502249586 %
MCC =  42.878231089195666 %
F1 SCORE =  62.85249898415278 %


In [ ]:
#Random Forest
# Grid de hiperparámetros a evaluar

parametros = {'n_estimators': [150],
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth'   : [4, 3, 10, 20],
              'criterion'   : ['gini', 'entropy']
             }

# Grid search , utlizando  validación cruzada 5 pliegues, 3 repeticiones 
grid = GridSearchCV(
        estimator  = RandomForestClassifier(random_state = 100),
        param_grid = parametros,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        #Vaidación cruzada, 5 pliegues 
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0), 
        #refit=True, el reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en .best_estimator_.
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )
grid.fit(PCA_X_train,y_train)
modelo_finall4 = grid.best_estimator_
predictiones = modelo_finall4.predict(PCA_X_test)

print("Los mejores hiperparámetros encontrados" + str(grid.best_params_) + " score: " + str(grid.best_score_*100),"%")
print('MCC = ',matthews_corrcoef(y_test,modelo_finall4.predict(PCA_X_test))*100,"%")
print("F1 SCORE = ",f1_score(y_test,predictiones,average='weighted')*100,"%")

Los mejores hiperparámetros encontrados{'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto', 'n_estimators': 150} score: 65.72484016102297 %
MCC =  52.11263470431171 %
F1 SCORE =  68.48578555672377 %
